In [19]:
import glob
from tqdm import tqdm
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from collections import Counter, defaultdict
from nltk import ngrams

[nltk_data] Downloading package punkt to /Users/arpan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/arpan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/arpan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/arpan/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
files = glob.glob('sampled_tweets/*[0-9].npy')
files.sort()

In [3]:
len(files)

222

In [4]:
tweet_text = []
reaction = []
for file in tqdm(files[:10]):
    tweets = np.load(file, allow_pickle=True).tolist()
    for tweet in tweets:
        if tweet['lang'] == 'en':
            text = tweet['full_text']
            if 'need' in text or 'needs' in text or 'want' in text or 'require' in text:
                tweet_text.append(text)
                reaction.append(int(0.01*tweet['retweet_count']) + int(0.005*tweet['favorite_count']))

100%|███████████████████████████████████████████| 10/10 [01:28<00:00,  8.87s/it]


In [5]:
print(len(tweet_text))

28789


## Unigrams

In [6]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words())
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /Users/arpan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
noise = set(["https", "@", "…", "Putin", "“", "’"])
unigram_counter = defaultdict(int)
noun = set(("NN", "NNS"))
for i, sentence in tqdm(enumerate(tweet_text), total=len(tweet_text)):
    pos = nltk.pos_tag(nltk.word_tokenize(sentence))
    for word, tag in pos:
        if tag in noun and word not in noise and len(word) > 3:
            unigram_counter[word] += reaction[i]

100%|████████████████████████████████████| 28789/28789 [00:30<00:00, 958.76it/s]


In [9]:
len(unigram_counter)

19633

In [10]:
unigram_counter = dict(sorted(unigram_counter.items(), key=lambda k_v: k_v[1], reverse=True))
unigram_counter

{'people': 450836.3200000022,
 'world': 376378.77999999985,
 'offices': 370687.4000000008,
 'night': 368471.6399999993,
 'picture': 368307.5800000009,
 'address': 368255.55000000086,
 'lawmakers': 368242.4900000009,
 'dictator': 366047.3999999996,
 'daughter': 365931.0699999996,
 'peace': 58813.13999999989,
 'nation': 54809.82999999986,
 'navalny': 54430.879999999845,
 'want': 54429.279999999846,
 'thread': 39534.589999999946,
 'oneunderscore__': 38197.31999999995,
 'blogger': 38195.64999999995,
 'land': 36392.679999999986,
 'maxseddon': 36106.499999999985,
 'neg…': 35243.72999999999,
 'need': 33786.709999999985,
 'sanctions': 29557.400000000016,
 'leadership': 27749.760000000017,
 'countries': 23932.409999999978,
 'ammunition': 23861.059999999965,
 'hours': 23382.05999999998,
 'homes': 22985.160000000025,
 'protection': 22622.189999999995,
 'ride': 22405.09999999996,
 'country': 21920.740000000013,
 'olgatokariuk': 20770.620000000035,
 'everyone': 19690.72999999999,
 'support': 18680.

## Bigrams

In [20]:
bigram_counter = defaultdict(int)
noun = set(("NN", "NNS"))
for i, sentence in tqdm(enumerate(tweet_text), total=len(tweet_text)):
    n = 2
    ngram = ngrams(sentence.split(), n)  
    for grams in ngram:
        pos = nltk.pos_tag(grams)
        if pos[0][1] in noun and pos[1][1] in noun:
            bigram_counter[" ".join([pos[0][0], pos[1][0]])] += reaction[i]

100%|████████████████████████████████████| 28789/28789 [01:07<00:00, 424.48it/s]


In [21]:
len(bigram_counter)

32597

In [22]:
bigram_counter = dict(sorted(bigram_counter.items(), key=lambda k_v: k_v[1], reverse=True))
bigram_counter

{'lawmakers: “I': 368225.4700000009,
 'need ammunition,': 23788.94999999997,
 'hours, countless': 22441.730000000007,
 'countless Ukrainians': 22441.730000000007,
 'need protection.': 22441.730000000007,
 'don’t think': 16696.450000000026,
 'everyone remembers': 16683.05000000003,
 'favor though”…': 16683.05000000003,
 'Ukraine right': 15048.61000000001,
 '@jegdorothea: #SanctionRussiaNow': 15045.52000000001,
 'saying: "I': 14197.999999999982,
 'street lit': 14028.169999999996,
 '🇹🇭 friends.': 13898.48,
 'stop showing': 13857.129999999986,
 '@AizelKonArt: 🇺🇦': 12440.029999999984,
 'art people,': 12440.029999999984,
 'promote amazing': 12440.029999999984,
 '&amp; girls:': 10747.61,
 'attention boys': 10736.25,
 'to: https://t.co/RKCwdm7RsR': 10586.939999999991,
 "speech, here's": 9587.120000000004,
 'glimpse: moms': 9587.120000000004,
 'Ukraine needs': 8703.119999999984,
 'class job': 7869.2999999999865,
 'support Ukraine': 7742.41000000002,
 'hate him?': 6408.889999999997,
 '“I don’t':

## NER

In [23]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text


  return [ent1.strip(), ent2.strip()]

In [24]:
ent_who = []
ent_what = []
noise = set(["Ukraine, Russia"])
ent_what_counter = defaultdict(int)
ent_who_counter = defaultdict(int)
for i, sentence in tqdm(enumerate(tweet_text), total=len(tweet_text)):
    ent1, ent2 = get_entities(sentence)
    if ent1 not in noise and ent2 not in noise:
        ent_who_counter[ent1] += reaction[i]
        ent_what_counter[ent2] += reaction[i]

100%|████████████████████████████████████| 28789/28789 [01:10<00:00, 408.24it/s]


In [25]:
ent_what_counter

defaultdict(int,
            {'international security guarantees': 193.32000000000005,
             'car': 0.8300000000000001,
             'private  wit': 10736.25,
             'o': 9605.290000000005,
             'world': 366282.5599999996,
             'invaded  terrorists': 0.0,
             'how  Putin': 1.76,
             'Putin  two': 4540.339999999994,
             'what': 5500.080000000005,
             'back  2018': 238.0,
             'control': 0.46,
             'even Kremlin quotes': 4.5,
             'Russia': 1568.6799999999996,
             'PutinsWar': 0.01,
             'gay  Ukraine': 0.01,
             'young Eastern bo': 0.28,
             'separatist regions https://t.co/wqaboyfai2': 0.19,
             'Ukraine': 9497.140000000005,
             'editorials': 0.0,
             'only cabin bag': 0.0,
             '': 2778.9500000000025,
             'entire  Soviet': 0.06,
             'matter': 566.81,
             'House': 376.7499999999999,
             'just s

In [26]:
ent_who_counter

defaultdict(int,
            {'that': 1609.2700000000004,
             'he': 6417.370000000011,
             'reduced  He': 10736.25,
             'here  moms': 9587.120000000004,
             'This': 777.2100000000003,
             'sanctioning': 0.0,
             'nah bro lemme': 1.76,
             'absolutely  him': 4540.339999999994,
             'you': 62114.949999999946,
             'what': 4568.520000000003,
             'well  they': 0.0,
             'Ukraine': 3916.08,
             'west': 24.0,
             'they': 1974.4399999999982,
             'tolerant  who': 0.01,
             'Venezuelan RT refugee': 0.28,
             'here  you': 6.819999999999999,
             'Donbas': 0.0,
             'who': 17165.189999999995,
             'self I': 0.52,
             'why  it': 1159.2799999999997,
             'John we': 195.52,
             'afraid  Republicans': 376.7499999999999,
             'Putin': 2140.649999999999,
             'We': 3517.5599999999945,
             '